In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.utils as nn_utils


In [2]:
train_combined_embeddings_tensor = torch.load('train_embeddings.pt')
val_combined_embeddings_tensor = torch.load('val_embeddings.pt')
test_combined_embeddings_tensor = torch.load('test_embeddings.pt')


In [3]:
# Reshape the train embeddings
train_combined_embeddings_tensor = train_combined_embeddings_tensor.unsqueeze(1)  # Shape: [185659, 1, 775]

# Assuming val_combined_embeddings_tensor is your validation embeddings tensor
val_combined_embeddings_tensor = val_combined_embeddings_tensor.unsqueeze(1)  # Shape: [val_size, 1, 775]

# Assuming test_combined_embeddings_tensor is your test embeddings tensor
test_combined_embeddings_tensor = test_combined_embeddings_tensor.unsqueeze(1)  # Shape: [test_size, 1, 775]


In [4]:
# Define your tensors here or load them from disk
train_labels_tensor = torch.load('train_labels_tensor.pt')
val_labels_tensor = torch.load('val_labels_tensor.pt')
test_labels_tensor = torch.load('test_labels_tensor.pt')

In [5]:
batch_size = 32

In [6]:
train_dataset = TensorDataset(train_combined_embeddings_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_combined_embeddings_tensor, val_labels_tensor)
test_dataset = TensorDataset(test_combined_embeddings_tensor, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [7]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [8]:
def adjust_labels(labels):
    return labels.float().unsqueeze(1)  # Change shape from [batch_size] to [batch_size, 1]


In [9]:


# Function to train the model
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        train_loss = 0.0
        all_labels=[]
        all_preds=[]
        for batch in train_loader:
            embeddings, labels = batch
            embeddings , labels = embeddings.to(device),adjust_labels(labels).to(device)
            labels = labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(embeddings).float()
            
            # Calculate loss
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()

            optimizer.step()
            
            train_loss += loss.item()

            # Collect predictions and labels for metric calculation
            preds = (outputs > 0.5).float()
            all_labels.append(labels.cpu().numpy())
            all_preds.append(preds.cpu().numpy())
        
        # Calculate metrics
        all_labels = np.concatenate(all_labels)
        all_preds = np.concatenate(all_preds)

        train_accuracy = accuracy_score(all_labels, all_preds)
        train_precision = precision_score(all_labels, all_preds)
        train_recall = recall_score(all_labels, all_preds)
        train_f1 = f1_score(all_labels, all_preds)

        # Print metrics for training
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss / len(train_loader)}')
        print(f'Training Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1 Score: {train_f1:.4f}')
              
        # Print the average training loss for this epoch
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss / len(train_loader)}')
        
        # Validate the model
        validate_model(model, val_loader, criterion,epoch)

# Function to validate the model
def validate_model(model, val_loader, criterion,epoch):
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for batch in val_loader:
            embeddings, labels = batch
            embeddings , labels = embeddings.to(device) ,adjust_labels(labels).to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(embeddings).float()
            
            # Calculate loss
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Collect predictions and labels for metric calculation
            preds = (outputs > 0.5).float()
            all_labels.append(labels.cpu().numpy())
            all_preds.append(preds.cpu().numpy())
    # Calculate metrics
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)

    val_accuracy = accuracy_score(all_labels, all_preds)
    val_precision = precision_score(all_labels, all_preds)
    val_recall = recall_score(all_labels, all_preds)
    val_f1 = f1_score(all_labels, all_preds)

    # Print metrics for validation
    print(f'Validation Loss: {val_loss / len(val_loader)}')
    print(f'Validation Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1 Score: {val_f1:.4f}')



    # Step the scheduler
    scheduler.step(val_loss)
    
  


In [10]:


class TransformerDecoderClassifier(nn.Module):
    def __init__(self, num_classes, embedding_dim=768, num_decoder_layers=6, dropout=0.1):
        super(TransformerDecoderClassifier, self).__init__()
        
        self.embedding_dim = embedding_dim
        
         # Linear layer to project embeddings to the desired dimension
        self.projection = nn.Linear(775, self.embedding_dim)
        
        # Define the Transformer decoder layer
        decoder_layer = nn.TransformerDecoderLayer(d_model=embedding_dim, nhead=8, dropout=dropout)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
        
        # Final linear layer for classification
        self.fc = nn.Linear(embedding_dim, num_classes)
        
        # Sigmoid for binary classification
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # x shape: [batch_size, seq_len, embedding_dim]
        # Project embeddings to the desired dimension
        x = self.projection(x)
        
        # Apply the Transformer decoder
        x = self.transformer_decoder(x, x)  # tgt and memory both are the input in this case
        
        # Apply the final linear layer and sigmoid activation
        x = self.fc(x[:, -1, :])  # Use the output from the last token
        
        return self.sigmoid(x)

# Initialize the model
model = TransformerDecoderClassifier(
    num_classes=1,  # Binary classification
    embedding_dim=768,  # Updated embedding dimension
    num_decoder_layers=6,  # Number of decoder layers
    dropout=0.1  # Dropout rate
).to(device)



In [11]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=1e-5)  # Adjust learning rate as needed
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True )


d:\python\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [12]:

# Assuming you have the model, optimizer, criterion, and data loaders ready:
num_epochs = 10  # Adjust based on your dataset size and needs
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)


d:\python\Lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch [1/10], Loss: 0.21399440768875527
Training Accuracy: 0.9164, Precision: 0.9168, Recall: 0.9159, F1 Score: 0.9164
Epoch [1/10], Loss: 0.21399440768875527
Validation Loss: 0.20831699556831657
Validation Accuracy: 0.9212, Precision: 0.9024, Recall: 0.9447, F1 Score: 0.9231
Epoch [2/10], Loss: 0.1991200460470181
Training Accuracy: 0.9226, Precision: 0.9225, Recall: 0.9228, F1 Score: 0.9226
Epoch [2/10], Loss: 0.1991200460470181
Validation Loss: 0.19575665648641258
Validation Accuracy: 0.9235, Precision: 0.9138, Recall: 0.9353, F1 Score: 0.9244
Epoch [3/10], Loss: 0.1954694988768452
Training Accuracy: 0.9245, Precision: 0.9244, Recall: 0.9247, F1 Score: 0.9245
Epoch [3/10], Loss: 0.1954694988768452
Validation Loss: 0.19789972008302295
Validation Accuracy: 0.9251, Precision: 0.9142, Recall: 0.9383, F1 Score: 0.9261
Epoch [4/10], Loss: 0.19243386905344217
Training Accuracy: 0.9257, Precision: 0.9257, Recall: 0.9257, F1 Score: 0.9257
Epoch [4/10], Loss: 0.19243386905344217
Validation Los

In [13]:


def test_model(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    all_labels = []
    all_preds = []

    with torch.no_grad():  # No gradient calculation during testing
        for embeddings, labels in test_loader:
            embeddings = embeddings.to(device)
            labels = adjust_labels(labels).to(device).float()  # Convert labels to float if using BCELoss


            outputs = model(embeddings)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Assuming outputs and labels are in the range [0, 1]
            preds = (outputs > 0.5).float()

            all_labels.append(labels.cpu().numpy())
            all_preds.append(preds.cpu().numpy())

    # Calculate average test loss
    test_loss /= len(test_loader)

    # Convert lists to numpy arrays
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f'Test Loss: {test_loss}')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')

    return test_loss, accuracy, precision, recall, f1


In [14]:
import numpy as np

In [15]:
# Assuming test_loader is defined, and you have the model and criterion set up
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Testing the model
test_loss, accuracy, precision, recall, f1 = test_model(model, test_loader, criterion)


Test Loss: 0.1849774606294673
Accuracy: 0.9285991462941404
Precision: 0.9424274781989678
Recall: 0.9130172413793104
F1 Score: 0.9274892722655224
